In [1]:
''' LIBRARIES '''
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

import re
import time
from IPython.display import clear_output
import multiprocessing
import threading
from tqdm import tqdm

from fuzzywuzzy import process, fuzz
from sympy import symbols, Eq, solve
import pandas as pd
import numpy as np

c:\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
from tqdm import tqdm

In [3]:
lista = []
for i in tqdm(range(700)):
    lista.append(i)

100%|██████████| 700/700 [00:00<?, ?it/s]


## Navigators

In [2]:
'''Navigator ADMIRAL PREMATCH'''
url_adm = 'https://admiralbet.rs/sport-prematch?sport=Fudbal'
options = Options()
options.headless = False #for not opening the navigator


adm = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
adm.get(url_adm)
adm.maximize_window()
cookie = adm.find_element(By.XPATH, '//*[@id="cookieContainer"]/div[3]/a/span')
cookie.click()

adm.switch_to.frame(adm.find_element(By.XPATH, '//*[@id="sportIframe"]'))

day = adm.find_element(By.XPATH, '//*[@id="centerContent"]/app-filter/section/div/div/div[5]')
day.click()

C:\Users\KNL\AppData\Local\Temp\ipykernel_4584\345610876.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  adm = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)


In [3]:
''' SOCCERBET NAVIGATOR '''
url_soc = 'https://soccerbet.rs/#kladjenje'


#changing chromedriver default options
options = Options()
options.headless = False #for not opening the navigator
options.add_argument('window-size=1920x1080') #Headless = True
soc = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
soc.get(url_soc)
soc.maximize_window()
cancel = soc.find_element(By.XPATH, '//*[@id="promotionPopupModal"]/div/div/div/div/input')
cancel.click()
day = WebDriverWait(soc, 4).until(EC.presence_of_element_located((By.XPATH, '//*[@id="home"]/div[1]/div[1]/ul/li[3]/a/span[1]')))
day.click()

C:\Users\KNL\AppData\Local\Temp\ipykernel_4584\3256336066.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  soc = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)


In [4]:
'''Navigator Options               StarBETa LIVE'''
url_star = 'https://www.starbet.rs/Bet'
options = Options()
options.headless = False #for not opening the navigator


star = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
star.get(url_star)
star.maximize_window()
cancel = WebDriverWait(star, 4).until(EC.presence_of_element_located((By.XPATH, '//*[@id="zatvarac"]')))
cancel.click()
checkmark_futbal = WebDriverWait(star, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="tmpSportoviLigi"]/div[2]/div[1]/h4/label/span')))
checkmark_futbal.click()
day = WebDriverWait(star, 3).until(EC.presence_of_element_located((By.XPATH, '//*[@id="tb459"]/tbody/tr[1]/td[1]/span')))
day.click()
day.click()


C:\Users\KNL\AppData\Local\Temp\ipykernel_4584\92520508.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  star = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)


In [5]:
'''Navigator MAXBET '''
url_max = 'https://www.maxbet.rs/ibet-web-client/#/home#top'
options = Options()
options.headless = False #for not opening the navigator


maxb = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
maxb.get(url_max)
maxb.maximize_window()
slider = maxb.find_element(By.XPATH, '//span[contains(@class, "slider-handle")]')
ActionChains(maxb).drag_and_drop_by_offset(slider, -80,0).perform()
#fudbal  = maxb.find_element(By.XPATH, '//*[@id="topView"]/div/div[2]/div[1]/div[2]/div[4]/div/div[1]/div[1]/div[2]/div[1]')
time.sleep(2)
fudbal = WebDriverWait(maxb, 4).until(EC.presence_of_element_located((By.XPATH, '//*[@id="topView"]/div/div[2]/div[1]/div[2]/div[4]/div/div[1]/div[1]/div[2]/div[1]')))
#fudbal.click()
fudbal.click()
time.sleep(1)

election = WebDriverWait(maxb, 4).until(EC.presence_of_element_located((By.XPATH, '//*[@id="topView"]/div/div[2]/div[1]/div[2]/div[4]/div/div[1]/div[2]/div[1]')))
#election  = maxb.find_element(By.XPATH, '//*[@id="topView"]/div/div[2]/div[1]/div[2]/div[4]/div/div[1]/div[2]/div[1]')
election.click()

C:\Users\KNL\AppData\Local\Temp\ipykernel_4584\462740259.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  maxb = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)


## Functions

In [41]:
def starbet_pre():
    df_star = pd.DataFrame(columns=['teams', '1_star', 'X_star', '2_star', 'under_star', 'goals_star', 'over_star'])
    for j in range(80):
        for k in range(1,40):
            try:
                elemento = star.find_element(By.XPATH, f'/html/body/form/div[4]/div/div/div/div[2]/div[6]/table[{j}]/tbody/tr[{k}]').text.split('\n')
                teams = elemento[1].replace(':', '')

                home = elemento[2][:14].split(' ')[0]
                tie  = elemento[2][:14].split(' ')[1]
                away = elemento[2][:14].split(' ')[2]

                goals = 2.5
                under = elemento[2][35:44].split(' ')[0]
                over  = elemento[2][35:44].split(' ')[1]
                df_star.loc[len(df_star.index)] = [teams, home, tie, away, under, goals, over]
            except:
                pass
        #clear_output()
       # porcentaje = (j+1)*10/8
       # print(f'starbet {porcentaje}%')
    df_star.to_csv('CSV\df_star.csv')
    #return print('df_star.csv updated')

In [42]:
def soccer_pre(pages=6):    
    df_soc = pd.DataFrame(columns=['teams', '1_soc', 'X_soc', '2_soc', 'under_soc', 'goals_soc', 'over_soc'])

    for j in range(pages):

        nex = WebDriverWait(soc, 3).until(EC.presence_of_element_located((By.XPATH, '//span[contains(@class, "triangle-right")]')))
        for k in range(40):
            try:
                elemento = soc.find_element(By.XPATH, f'//*[@id="result-tables"]/div[2]/div[2]/div/table/tbody/tr[{k}]').text.split('\n')
                teams = elemento[1][4:]
                
                home = elemento[2]
                tie = elemento[3]
                away = elemento[4]

                under = elemento[5]
                goals = 2.5
                over = elemento[6]
            
                df_soc.loc[len(df_soc.index)] = [teams, home, tie, away, under, goals, over]
            except:
                pass
        nex.click()
    time.sleep(2)
    first = WebDriverWait(soc, 3).until(EC.presence_of_element_located((By.XPATH, '//span[contains(@class, "fast-backward")]')))
    first.click()
    first.click()
    #clear_output()
    #print('soccer_pre done')
    df_soc.to_csv('CSV\df_soc.csv')
    return df_soc

In [43]:
def admiral_pre(pages=7):
    df_adm = pd.DataFrame(columns=['teams', '1_adm', 'X_adm', '2_adm', 'under_adm', 'goals_adm', 'over_adm'])

    nex = WebDriverWait(adm, 4).until(EC.presence_of_element_located((By.XPATH, '//ul/li[@class="page-item"][1]')))
    for j in range(pages):
        for k in range(40):
            try:
                elemento = adm.find_element(By.XPATH, f'//*[@id="centerContent"]/app-sport-tree-selection-container/div/app-events-group/section/section/div/app-event[{k}]').text.split('\n')
                teams = elemento[3]+' '+elemento[4]

                home=elemento[5]
                tie =elemento[6]
                away=elemento[7]

                under=elemento[8]
                goals = elemento[9]
                over=elemento[10]
                df_adm.loc[len(df_adm.index)] = [teams, home, tie, away, under, goals, over]

            except:
                pass
        nex.click()
        nex = WebDriverWait(adm, 4).until(EC.presence_of_element_located((By.XPATH, '//ul/li[@class="page-item"][2]')))

    first = WebDriverWait(adm, 4).until(EC.presence_of_element_located((By.XPATH, '//li[contains(@class, "page-item ng-star-inserted")][1]')))
    first.click()
    df_adm.to_csv('CSV\df_adm.csv')
    #print('admiral_pre done')
    return df_adm

In [44]:
def maxbet_pre():
    df_max = pd.DataFrame(columns=['teams', '1_max', 'X_max', '2_max', 'under_max', 'goals_max', 'over_max'])
    for j in tqdm(range(60)):
        for k in range(10):
            try:
                elemento = maxb.find_element(By.XPATH, f'/html/body/div[1]/div[5]/div/div[2]/div[2]/div/div/div/div[2]/div[{j}]/league-with-matches/div/div[{k}]').text.split('\n')
                teams = elemento[2].replace('-','')

                home = elemento[3]
                tie = elemento[4]
                away = elemento[5]

                under = elemento[6]
                goals = 2.5
                over = elemento[7]
                df_max.loc[len(df_max.index)] = [teams, home, tie, away, under, goals, over]

            except:
                pass
        porcentaje=round((j+1)*10/6, 3)
        #clear_output()
        #print(f'maxbet {porcentaje}%')
    df_max.to_csv('CSV\df_max.csv')
    

In [45]:
def merge():
    '''          STARBET / ADMIRALBET '''

    df_star, df_adm = pd.read_csv('CSV\df_star.csv') , pd.read_csv('CSV\df_adm.csv')

    teams_adm  = df_adm.teams.tolist()
    teams_star = df_star.teams.tolist()

    df_adm[['teams_matched', 'score']] = df_adm.teams.apply(lambda x:process.extractOne(x, teams_star, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
    df_adm = df_adm[df_adm.score>60]

    df_star_adm = pd.merge(df_adm, df_star, left_on='teams_matched', right_on='teams', how='left')
    df_star_adm.to_csv('CSV\TwoWay\PRE_star_adm.csv')


    '''          ADMIRALBET / SOCCERBET ''' 

    df_adm, df_soc = pd.read_csv('CSV\df_adm.csv'), pd.read_csv('CSV\df_soc.csv')

    teams_adm = df_adm.teams.tolist()
    teams_soc = df_soc.teams.tolist()

    df_soc[['teams_matched', 'score']] = df_soc.teams.apply(lambda x: process.extractOne(x, teams_adm, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
    df_soc = df_soc[df_soc.score>50]

    df_soc_adm = pd.merge(df_soc, df_adm, left_on='teams_matched', right_on='teams', how='left')
    df_soc_adm.to_csv('CSV\TwoWay\PRE_soc_adm.csv')

    '''         STARBET / SOCCERBET ''' 

    df_star, df_soc = pd.read_csv('CSV\df_star.csv'), pd.read_csv('CSV\df_soc.csv')

    teams_star = df_star.teams.tolist()
    teams_soc = df_soc.teams.tolist()

    df_soc[['teams_matched', 'score']] = df_soc.teams.apply(lambda x: process.extractOne(x, teams_star, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
    df_soc = df_soc[df_soc.score>50]

    df_soc_star = pd.merge(df_soc, df_star, left_on='teams_matched', right_on='teams', how='left')
    df_soc_star.to_csv('CSV\TwoWay\PRE_soc_star.csv') 


    '''          MAXBET / SOCCERBET '''

    df_soc, df_max = pd.read_csv('CSV\df_soc.csv'), pd.read_csv('CSV\df_max.csv')

    teams_soc = df_soc.teams.tolist()
    teams_max = df_max.teams.tolist()

    df_soc[['teams_matched', 'score']] = df_soc.teams.apply(lambda x:process.extractOne(x, teams_max, scorer= fuzz.token_sort_ratio)).apply(pd.Series)
    df_soc = df_soc[df_soc.score>59]

    df_soc_max = pd.merge(df_soc, df_max, left_on='teams_matched', right_on='teams', how='left')
    df_soc_max.to_csv('CSV\TwoWay\PRE_soc_max.csv')

    '''          MAXBET / ADMIRAL ''' 

    df_adm, df_max = pd.read_csv('CSV\df_adm.csv'), pd.read_csv('CSV\df_max.csv')

    teams_adm = df_adm.teams.tolist()
    teams_max = df_max.teams.tolist()

    df_adm[['teams_matched', 'score']] = df_adm.teams.apply(lambda x:process.extractOne(x, teams_max, scorer= fuzz.token_sort_ratio)).apply(pd.Series)
    df_adm = df_adm[df_adm.score>59]

    df_adm_max = pd.merge(df_adm, df_max, left_on='teams_matched', right_on='teams', how='left')
    df_adm_max.to_csv('CSV\TwoWay\PRE_adm_max.csv')


    ''' 3WAY MERGES''' 


    ''' AD / ST / SC '''
    # loading data
    df_soc_adm = pd.read_csv('CSV\TwoWay\PRE_soc_adm.csv')
    df_soc_adm.drop(columns=['Unnamed: 0', 'Unnamed: 0_x','Unnamed: 0_y','teams_x', 'teams_y', 'score' ], inplace=True)
    df_star = pd.read_csv('CSV\df_star.csv')
    #Start Merging process
    teams_star = df_star.teams.tolist()
    df_soc_adm[['teams_matched2', 'score']] = df_soc_adm.teams_matched.apply(lambda x:process.extractOne(x, teams_star, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
    df_soc_adm = df_soc_adm[df_soc_adm.score>59] #This drops all the teams names that don't match among sites
    #The last merging process
    df_soc_adm_star = pd.merge(df_soc_adm, df_star, left_on='teams_matched2', right_on='teams', how='left')
    df_soc_adm_star.to_csv('CSV\ThreeWay\Pre_soc_adm_star.csv')
    


    ''' AD / SC / MAX '''
    # loading data
    df_soc_adm = pd.read_csv('CSV\TwoWay\PRE_soc_adm.csv')
    df_soc_adm.drop(columns=['Unnamed: 0', 'Unnamed: 0_x','Unnamed: 0_y','teams_x', 'teams_y', 'score' ], inplace=True)
    df_max = pd.read_csv('CSV\df_max.csv')
    #Start Merging process
    teams_max = df_max.teams.tolist()
    df_soc_adm[['teams_matched2', 'score']] = df_soc_adm.teams_matched.apply(lambda x:process.extractOne(x, teams_max, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
    df_soc_adm = df_soc_adm[df_soc_adm.score>59] #This drops all the teams names that don't match among sites
    #The last merging process
    df_soc_adm_max = pd.merge(df_soc_adm, df_max, left_on='teams_matched2', right_on='teams', how='left')
    df_soc_adm_max.to_csv('CSV\ThreeWay\Pre_soc_adm_max.csv')
    


    ''' SC / ST / MAX '''
    df_soc_star = pd.read_csv('CSV\TwoWay\PRE_soc_star.csv')
    df_soc_star.drop(columns=['Unnamed: 0', 'Unnamed: 0_x','Unnamed: 0_y','teams_x', 'teams_y', 'score' ], inplace=True)

    teams_max = df_max.teams.tolist()
    df_soc_star[['teams_matched2', 'score']] = df_soc_star.teams_matched.apply(lambda x:process.extractOne(x, teams_max, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
    df_soc_star = df_soc_star[df_soc_star.score>59] #This drops all the teams names that don't match among sites
    #The last merging process
    df_soc_star_max = pd.merge(df_soc_star, df_max, left_on='teams_matched2', right_on='teams', how='left')
    df_soc_star_max.to_csv('CSV\ThreeWay\Pre_soc_star_max.csv')
    

In [46]:
def final_function(rango=3):
    for i in range(rango):
        clear_output()
        admiralbet = threading.Thread(target=admiral_pre)
        soccerbet = threading.Thread(target=soccer_pre)
        #starbet = threading.Thread(target=starbet_pre)
        maxbet = threading.Thread(target=maxbet_pre)

        print(f'Iteration number: {i+1}')
        admiralbet.start()
        print('-------')
        print('Admiral function started')

        soccerbet.start()
        print('-------')
        print('Soccerbet function started')
        #starbet.start()
        maxbet.start()
        print('-------')
        print('Maxbet function started')
        admiralbet.join()
        print('-------')
        print('Admiral function ended')

        soccerbet.join()
        print('-------')
        print('Soccerbet function ended')
        
        #starbet.join()
        #print('-------')
        #print('Starbet function ended')
        
        maxbet.join()
        print('-------')
        print('Maxbet function ended')
        print('-------')
        print('Merging start')
        merge()
        print('-------')
        print('Merging finished. All .csv files have been updated')
        time.sleep(3)
    
    #print(f'iteration {i}/{tiempo}')

In [49]:
final_function(2)

Iteration number: 2
-------
Admiral function started
-------
Soccerbet function started
-------
Maxbet function started


  7%|▋         | 4/60 [00:01<00:19,  2.90it/s]Exception in thread Thread-88 (soccer_pre):
Traceback (most recent call last):
  File "c:\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\KNL\AppData\Local\Temp\ipykernel_4584\1241145211.py", line 23, in soccer_pre
  File "c:\Python310\lib\site-packages\selenium\webdriver\remote\webelement.py", line 88, in click
    self._execute(Command.CLICK_ELEMENT)
  File "c:\Python310\lib\site-packages\selenium\webdriver\remote\webelement.py", line 396, in _execute
    return self._parent.execute(command, params)
  File "c:\Python310\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 428, in execute
    self.error_handler.check_response(response)
  File "c:\Python310\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 243, in check_response
    raise exception_class(message, scree

-------
Admiral function ended
-------
Soccerbet function ended


100%|██████████| 60/60 [00:22<00:00,  2.67it/s]


-------
Maxbet function ended
-------
Merging start
-------
Merging finished. All .csv files have been updated
